In [3]:
# %%
import pennylane as qml
from pennylane import numpy as np
import covalent as ct
import os

import dotenv
dotenv.load_dotenv()
IBQM_token = os.getenv('IBMQ_token')

# %%
# HYPERPARAMETERS
NOISE_MODEL = "IBM"
n_qubits = 2
n_shots = 10**4
NUM_TRIALS = 1

# create our devices to run our circuits on
dev_mu = qml.device("default.mixed", wires=n_qubits+1)
dev_x = qml.device("default.mixed", wires=n_qubits, shots=n_shots)

if NOISE_MODEL == "IBM":
    # Define a Qiskit Runtime executor.
    qiskit_executor = ct.executor.QiskitExecutor(
        device="sampler",
        backend="ibmq_qasm_simulator",
        ibmqx_token=IBQM_token,  # required if not in config file
        single_job=False,
    )

# %%
@ct.qelectron(executors=qiskit_executor)
@qml.qnode(dev_mu, interface="autograd")
def local_hadamard_test(l, lp, part=None):
    qml.Hadamard(wires=l)

    if part == "Im" or part == "im":
        qml.PhaseShift(-np.pi / 2, wires=l)

    qml.CNOT(wires=(l, lp))

    # Expectation value of Z for the ancillary qubit.
    return qml.expval(qml.PauliZ(wires=l))

# Computes the mu coefficients
@ct.electron
def mu(l=None, lp=None):
    """Generates the coefficients to compute the "local" cost function C_L."""
    mu_real = local_hadamard_test(l, lp, part="Re")
    mu_imag = local_hadamard_test(l, lp, part="Im")

    return mu_real + 1.0j * mu_imag

@ct.electron
def cost_loc(n_qubits):
    mu_sum = 0.0

    for l in range(0, n_qubits):
        for lp in range(0, n_qubits):
            if l != lp:
                mu_sum = mu_sum + mu(l, lp)

    mu_sum = abs(mu_sum)

    # Cost function C_L
    res = 0.5 - 0.5 * mu_sum
    return res

@ct.electron
def train_func(n_qubits):
    cost = cost_loc(n_qubits)

    return cost

# %%
@ct.lattice
def workflow(n_qubits = 2, error =  1.6 * 10**-3, n_shots=10**4):
    cost = train_func(n_qubits)

    return cost

# %%
for trial in range(NUM_TRIALS):
    dispatch_id = ct.dispatch(workflow)()
    res = ct.get_result(dispatch_id=dispatch_id, wait=True)
    print(res)

#%%
res.result


Lattice Result
status: COMPLETED
result: 0.4433088856496094
input args: []
input kwargs: {}
error: None

start_time: 2024-05-05 20:31:42.101724
end_time: 2024-05-05 20:34:30.227486

results_dir: /Users/bigsad/.local/share/covalent/data
dispatch_id: 1e93cbda-0524-48b2-b15c-face50dc1912

Node Outputs
------------
train_func(0): 0.4433088856496094
:parameter:2(1): 2
:postprocess:(2): 0.4433088856496094



tensor(0.44330889, requires_grad=True)